<a href="https://colab.research.google.com/github/kolovratgas/HPC_lab1/blob/main/Numba1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numba
!pip install --upgrade tbb

     |████████████████████████████████| 4.0 MB 5.4 MB/s 


In [2]:
from numba import njit, jit, vectorize, float64, cuda, prange
import numba
import numpy as np
import time
import pandas


%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['float64', 'vectorize']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
numba.cuda.detect()

Found 1 CUDA devices
id 0            b'Tesla K80'                              [SUPPORTED]
                      compute capability: 3.7
                           pci device id: 4
                              pci bus id: 0
Summary:
	1/1 devices are supported


True

In [4]:
x1 = 256
a1 = np.random.randn(x1, x1)
b1 = np.random.randn(x1, x1)
c1 = np.zeros((x1, x1))

In [ ]:
x2 = 512
a2 = np.random.randn(x2, x2)
b2 = np.random.randn(x2, x2)
c2 = np.zeros((x2, x2))

In [ ]:
x3 = 1024
a3 = np.random.randn(x3, x3)
b3 = np.random.randn(x3, x3)
c3 = np.zeros((x3, x3))

In [5]:
def cpuMM(a, b, c):
      for i in range(c.shape[0]):
        for j in range(c.shape[1]):               
            tmp = 0
            for k in range(a.shape[1]):
                tmp += a[i, k] * a[k, j] 
            c[i, j] = tmp 
host_mat1 = c1

In [6]:
startCPU1 = time.time()
cpuMM(a1,b1,c1)
endCPU1 = time.time()-startCPU1
print(endCPU1)

10.380768537521362


In [ ]:
startCPU2 = time.time()
cpuMM(a2,b2,c2)
endCPU2 = time.time()-startCPU2
print(endCPU2)

86.52684473991394


In [ ]:
startCPU3 = time.time()
cpuMM(a3,b3,c3)
endCPU3 = time.time()-startCPU3
print(endCPU3)

694.7470121383667


In [8]:
@cuda.jit
def gpuMM(a, b, c):
  i, j = cuda.grid(2)
  if i < c.shape[0] and j < c.shape[1]:
    tmp = 0
    for k in range(a.shape[1]):
      tmp += a[i, k] * b[k, j]
    c[i, j] = tmp

count_threads = 6

def DeviceHost(a, b, c):
  device_A = cuda.to_device(a)
  device_B = cuda.to_device(b)
  device_C = cuda.device_array(c.shape, np.float64)

  threads_per_block = (count_threads, count_threads)
  blocks_per_grid_x = math.ceil(a.shape[0]/threads_per_block[0])
  blocks_per_grid_y = math.ceil(a.shape[1]/threads_per_block[1])
  blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y)

  gpuMM[blocks_per_grid, threads_per_block](device_A, device_B, device_C)
  return device_C.copy_to_host()
dev_mat = c1


In [9]:
startGPU1 = time.time()
DeviceHost(a1,b1,c1)
endGPU1 = time.time()-startGPU1
print(endGPU1)

0.9079232215881348


In [10]:
startGPU2 = time.time()
DeviceHost(a2,b2,c2)
endGPU2 = time.time()-startGPU2
print(endGPU2)

NameError: ignored

In [ ]:
startGPU3 = time.time()
DeviceHost(a3,b3,c3)
endGPU3 = time.time()-startGPU3
print(endGPU3)

0.1592714786529541


In [11]:
print(np.allclose(dev_mat, host_mat1))

True


In [ ]:
timing_df = pandas.DataFrame({'name':['cpu', 'gpu', 'cpu', 'gpu', 'cpu', 'gpu'], 'timing_in_seconds':[endCPU1, endGPU1, endCPU2, endGPU2, endCPU3, endGPU3], 'matrix_size':['256 x 256', '256 x 256', '512 x 512', '512 x 512', '1024 x 1024', '1024 x 1024']})
timing_df

,name,timing in seconds,matrix size
0,cpu,10.783913,256 x 256
1,gpu,0.007460,256 x 256
2,cpu,86.526845,512 x 512
3,gpu,0.034464,512 x 512
4,cpu,694.747012,1024 x 1024
5,gpu,0.159271,1024 x 1024
